In [2]:
#@title ## Download Kaggle Dataset
#@markdown Dataset: Annotated Corpus for Named Entity Recognition <br>
#@markdown [https://www.kaggle.com/therohk/million-headlines](https://www.kaggle.com/therohk/million-headlines)
#@markdown <br><br>
#@markdown News headlines published over a period of seventeen years.

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d therohk/million-headlines
!unzip -o /content/million-headlines

million-headlines.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/million-headlines.zip
  inflating: abcnews-date-text.csv   
